In [1]:
# %pip install chromadb tiktoken

In [1]:
import os

from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import AutoModel
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


from gdoc_sync.integrations.udemy import RedundantFilterRetriever

# Retrieve from Chroma

- define llm for handling retrieved documents
- define model for handling embeddings
- instantiate vectorstore to retrieve data from
- chain for data retrieval

In [3]:
base_url = "http://10.0.0.56:7860/"
base_url = "http://172.21.240.1:1234/v1"

chat = ChatOpenAI(temperature=0, base_url=base_url, openai_api_key="not_needed")

In [4]:
model_name = "jinaai/jina-embeddings-v2-base-en"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
)

In [8]:
db = Chroma(persist_directory="./storage", embedding_function=embeddings)

# retriever = db.as_retriever()
retriever = RedundantFilterRetriever(embeddings=embeddings, chroma=db)

In [9]:
chain = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff",
    # chain_type="map_reduce",
    # chain_type = "map_rerank",
    # chain_type = "refine",
    verbose=True,
)

In [10]:
result = chain.run("what is an interesting fact about the english language?")
result



> Entering new RetrievalQA chain...

> Finished chain.


'Dreamt is the only English word that ends with the letters "mt."'

In [ ]:


os.environ["OPENAI_API_KEY"] = "OPENAI API KEY"

# initializing the embeddings
embeddings = OpenAIEmbeddings()

# default model = "gpt-3.5-turbo"
llm = ChatOpenAI()

directory = "YOUR DOCUMENTS PATH"

def load_docs(directory):
    loader = DirectoryLoader(directory)
    documents = loader.load()
    return documents

documents = load_docs(directory)

def split_docs(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)

db = Chroma.from_documents(
    documents=docs, 
    embedding=embeddings
)

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    similar_docs = db.similarity_search(query, k=2) # get two closest chunks
    answer = chain.run(input_documents=similar_docs, question=query)
    return answer
    
print("Private Q&A chatbot")
prompt = input("Enter your query here: ")

if prompt:
    answer = get_answer(prompt)
    print(f"Answer: {answer}")   